ESTA NOTEBOOK TRABAJA SOBRE UN POWERPOINT. EXTRAE LA INFORMACIÓN DEL TEXTO Y DEVUELVE UN JSON
Cada slide del powerpoint tendrá su número y sus campos de texto. Cada "cuadro de texto" dentro del powerpoint es un str.
Como parámetros, hay que especificar la ruta hacia el .pptx y la ruta donde querramos guardar el output.

input = path/archivo.pptx
output = path (GUARDA UN .JSON)

In [1]:
from pptx import Presentation
import os
import json

In [2]:
def extract_text_and_hyperlinks_from_shape(shape):
    """Extract text and hyperlinks from a single shape."""
    combined_texts = []

    if shape.has_text_frame:
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                text = run.text.strip()
                if run.hyperlink and run.hyperlink.address:
                    text = f"{text} (link: {run.hyperlink.address})"  # Include hyperlink in text
                combined_texts.append(text)
    elif shape.shape_type == 19:  # Table shape
        for row in shape.table.rows:
            for cell in row.cells:
                for paragraph in cell.text_frame.paragraphs:
                    for run in paragraph.runs:
                        text = run.text.strip()
                        if run.hyperlink and run.hyperlink.address:
                            text = f"{text} (link: {run.hyperlink.address})"  # Include hyperlink in text
                        combined_texts.append(text)
    elif shape.shape_type == 5:  # Group shape
        for s in shape.shapes:
            text = extract_text_and_hyperlinks_from_shape(s)
            combined_texts.append(text)

    # Remove any empty texts
    combined_texts = [text for text in combined_texts if text]

    return " ".join(combined_texts).strip()

def extract_pptx_to_json(pptx_path, output_folder):
    # Load the presentation
    prs = Presentation(pptx_path)
    presentation_data = []

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through the slides
    for slide_number, slide in enumerate(prs.slides, start=1):
        slide_data = {
            "slide_number": slide_number,
            "texts": []
        }

        # Iterate through shapes on each slide
        for shape in slide.shapes:
            text = extract_text_and_hyperlinks_from_shape(shape)
            if text:  # Only add non-empty texts
                slide_data['texts'].append(text)

        # Remove empty lists if they have no values
        if not slide_data['texts']:
            del slide_data['texts']

        # Add slide data to presentation data
        presentation_data.append(slide_data)

    # Save presentation data as JSON
    output_json_path = os.path.join(output_folder, os.path.basename(pptx_path).replace('.pptx', '.json'))
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(presentation_data, json_file, ensure_ascii=False, indent=4)

    print(f"Presentation processed and saved to {output_json_path}")

In [5]:
"""ESPECIFICAR LA RUTA AL ARCHIVO .PPTX DE INTERÉS (INPUT_FOLDER)
ESPECIFICAR LA CARPETA DONDE QUERRAMOS GUARDAR EL OUTPUT (EL .JSON) COMO OUTPUT_FOLDER"""
input_folder = r'../Data/2024-I+D_Ficha Registro Iniciativas.pptx'
output_folder = r'../Data'

extract_pptx_to_json(input_folder, output_folder)

Presentation processed and saved to ../Data\2024-I+D_Ficha Registro Iniciativas.json
